<a href="https://colab.research.google.com/github/dksifoua/Question-Answering/blob/master/1%20-%20Document%20reader%20Question%20Answering%20-%20DrQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Oct  9 01:09:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Dependencies

In [2]:
!pip install tqdm --upgrade >> /dev/null 2>&1
!pip install torchtext --upgrade >> /dev/null 2>&1
!pip install spacy --upgrade >> /dev/null 2>&1
!python -m spacy download en >> /dev/null 2>&1

In [9]:
import json
import tqdm
import spacy
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Dataset, Example, Field
from torchtext.data.iterator import BucketIterator
from torchtext.data.metrics import bleu_score

In [10]:
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

SEED = 546
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

Device: cuda


## Prepare data

In [11]:
%%time
!mkdir ./data

!wget --no-check-certificate \
    https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
    -O ./data/train.json

!wget --no-check-certificate \
    https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json \
    -O ./data/valid.json

mkdir: cannot create directory ‘./data’: File exists
--2020-10-09 01:12:02--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘./data/train.json’

./data/train.json   100%[===================>]  40.17M  79.5MB/s    in 0.5s    

2020-10-09 01:12:03 (79.5 MB/s) - ‘./data/train.json’ saved [42123633/42123633]

--2020-10-09 01:12:03--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [applicati

In [12]:
with open('./data/train.json', 'r', encoding='utf-8') as file:
    train = json.load(file)['data']

with open('./data/valid.json', 'r', encoding='utf-8') as file:
    valid = json.load(file)['data']

print(f'Length of train data: {len(train):,}')
print(f'Length of valid data: {len(valid):,}')

Length of train data: 442
Length of valid data: 35


In [13]:
def parse_json(data):
    result = []
    for paragraphs in data:
        for paragraph in paragraphs['paragraphs']:
            context = paragraph['context']
            for qas in paragraph['qas']:
                id = qas['id']
                question = qas['question']
                for answers in qas['answers']:
                    answer_start = answers['answer_start']
                    answer = answers['text']
                    result.append({
                        'id': id,
                        'context': context,
                        'question': question,
                        'answer_start': answer_start,
                        'answer': answer
                    })
    return result

train = parse_json(train)
valid = parse_json(valid)
print(f'Length of train qa pairs: {len(train):,}')
print(f'Length of valid qa pairs: {len(valid):,}')

Length of train qa pairs: 86,821
Length of valid qa pairs: 20,302


In [16]:
print('Question:', train[0]['question'])
print('Answer:', train[0]['answer'])
print('Answer start at:', train[0]['answer_start'])
train[0]['context']

Question: When did Beyonce start becoming popular?
Answer: in the late 1990s
Answer start at: 269


'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [22]:
train[0]['context'][269:269+17]

'in the late 1990s'